# importing packages

In [1]:
# pip install --upgrade torch torchvision
# use this for training

In [2]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU clean-text[gpl]

!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

!pip install faiss-cpu
!pip install hazm

     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 1.8MB 23.6MB/s 
     |████████████████████████████████| 1.2MB 56.0MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
     |████████████████████████████████| 890kB 44.8MB/s 
     |████████████████████████████████| 133kB 18.4MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 245kB 23.1MB/s 
rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.
     |████████████████████████████████| 8.2MB 17.2MB/s 
     |████████████████████████████████| 317kB 19.1MB/s 
     |████████████████████████████████| 235kB 56.6MB/s 
     |████████████████████████████████| 1.4MB 49.7MB/s 
  Created wheel for libwapiti: filename=l

In [3]:
import numpy as np 
import pandas as pd
import re
import os
import faiss
import hazm
from hazm import stopwords_list
import pickle
import requests
from termcolor import colored
import hazm
from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses
import torch
import tensorflow as tf
from sentence_transformers import models, SentenceTransformer, util, evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics.pairwise import cosine_similarity

SentenceTransformer_TrainedOn_FarsTail_Path = 'm3hrdadfi/bert-fa-base-uncased-farstail-mean-tokens'
SentenceTransformer_TrainedOn_wikinli_Path = 'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens'
SentenceTransformer_TrainedOn_wikitriplet_Path = 'm3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens'

# The Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'

df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)

# preparing inputs for semantic

corpora = []
for i in range(len(list_t)):

    keys = '[SEP]'.join(keywords[i])
    corpora.append(' '.join([list_t[i], keys]))


keys = []
for i in range(len(list_t)):
    keys.append('[SEP]'.join(keywords[i]))

bm_selected = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Results/BM25_PrimaryRanking.pkl')

# annotations

df_annotations = pd.read_csv('/content/drive/MyDrive/all_annotations.csv')
df_annotations.loc[:, 'score'] = df_annotations.loc[:, 'score'].apply(lambda x: x-1)
print(f'minimum score: {min(df_annotations.score)} and maximum: {max(df_annotations.score)}')

minimum score: 0.0 and maximum: 4.0


In [6]:
corpora = []
for i in range(len(list_t)):

    keys = '[SEP]'.join(keywords[i])
    corpora.append(' '.join([list_t[i], keys]))

# Helper

In [7]:

# phrases that need to be removed from titles
corona_phrases = ['کرونایی', 'کروناست' ,'کرونا', 'شیوع', 'بحران', 'ویروس',
                  'ویروس جدید', 'coronavirus', 'corona', 'کووید-19 ', 
                  'کووید', 'بیماری', 'بیمارانی', 'بیماران', '-۱۹', ' وی ', '19', '۱۹',
                  ' بیمار ', 'كرونا', 'كوويد', 'ويروس', r'(\s+)',]


normalizer = hazm.Normalizer()

def clean(text):
    """Cleans the titles for the semantic models"""
    for pattern in corona_phrases:
        text = re.sub(pattern, " ", text)

    text = re.sub(' +[\w] +', " ", text)
    text = normalizer.normalize(text)

    return text


#---------------------------------- get the results for this model-----------------------#

def get_resutls(questions, top_n):
    
    results = []
    for question in questions:

        print(question)
        # print('question type', type(question))


        # we give the cleaned question to the semantic model 
        question_cleaned = clean(question)

        question_emb = sentence_bert_embedder.encode(question_cleaned,
                                    convert_to_tensor=False,
                                    show_progress_bar = False)
        
        emb_que = np.array([question_emb])
        faiss.normalize_L2(emb_que)

        top_k = index.search(emb_que, top_n)
        indices = []
        scores = []

        # saving all the reults in a dictionary
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            
            indices.append(idx)
            scores.append(score)

        results.append({'question':question,
                        'index':indices,
                        'score':scores})
    return results

# The model


In [8]:
model = SentenceTransformer(SentenceTransformer_TrainedOn_wikinli_Path)

Exception when trying to download https://sbert.net/models/m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens.zip not found. Try to create it from scratch
Try to create Transformer Model m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens with mean pooling


# Fine Tuning Stage

In [9]:
epochs = 1
warm_up_s = 50
number_of_records=300

In [10]:


df_annotations_train = df_annotations.copy()[:number_of_records] # only a part of dataframe

labels_for_kfold = df_annotations_train.score.apply(lambda x: round(x))# for letting sklearn understand different labels


df_annotations_train.loc[:, 'score'] = df_annotations_train.score.apply(lambda x:x/4) # for cosine similarity




In [11]:

k = 3
kfold = StratifiedKFold(n_splits = k, shuffle = True)

for i, (train_idx, val_idx) in enumerate(kfold.split(df_annotations_train.sentence2, labels_for_kfold), 1):
    print(colored(f'fold {i}', 'blue'))
    epoch_evaluation = {}
    print(train_idx)

    # then each in an example
    training_examples = []
    for _, record in df_annotations_train.loc[train_idx].iterrows():
        training_examples.append(InputExample(texts = [str(record.sentence1),
                                                       str(record.sentence2)],
                                               label= np.float(record.score)))


    # using dataloader to give data to our model
    training_dataloader = DataLoader(training_examples,
                                    shuffle = True,
                                    batch_size = 128)

    loss_ = losses.CosineSimilarityLoss(model)

    # print(df_annotations_train.loc[val_idx, 'sentence1'])


    evaluator = evaluation.EmbeddingSimilarityEvaluator(df_annotations_train.loc[val_idx, 'sentence1'].values.tolist(),
                                                        df_annotations_train.loc[val_idx, 'sentence2'].values.tolist(),
                                                        df_annotations_train.loc[val_idx, 'score'].values.tolist())


    %time
    model.fit(
        train_objectives=[(training_dataloader, loss_)],
        epochs=epochs,
        evaluator=evaluator, 
        #warmup_steps=warm_up_s, 
        # write_csv =True
        )

fold 1
[  0   1   2   3   4   5   6   7  10  11  14  15  16  19  21  22  24  26
  27  28  29  30  31  32  33  34  36  37  39  40  41  42  46  47  48  50
  52  53  54  55  57  59  60  61  64  67  68  69  73  75  76  78  79  82
  83  85  87  89  90  91  92  93  96  97  98  99 102 104 106 107 108 109
 110 112 114 116 117 119 120 121 122 124 125 127 128 129 131 133 134 135
 136 137 141 142 143 144 146 148 151 152 153 155 156 159 161 162 166 169
 171 172 173 175 176 177 178 180 182 183 184 186 187 188 190 192 194 196
 197 198 200 201 203 204 206 207 208 209 210 212 213 214 215 216 217 218
 222 224 225 227 229 230 231 233 234 236 237 239 240 242 244 245 246 248
 249 250 252 255 256 257 258 259 260 261 263 264 265 266 267 268 269 271
 272 273 274 275 276 277 278 279 280 281 282 283 285 286 287 292 294 295
 297 299]
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs




fold 2
[  0   1   2   4   5   8   9  11  12  13  14  15  16  17  18  19  20  23
  24  25  26  27  29  32  34  35  36  38  39  40  42  43  44  45  46  47
  49  50  51  53  54  56  57  58  60  61  62  63  65  66  67  70  71  72
  74  77  79  80  81  84  86  87  88  94  95  97  98  99 100 101 103 104
 105 109 110 111 112 113 114 115 116 118 119 120 123 124 125 126 127 129
 130 132 136 137 138 139 140 141 142 143 145 147 149 150 153 154 155 156
 157 158 159 160 161 162 163 164 165 166 167 168 170 172 174 176 178 179
 181 183 185 189 190 191 192 193 194 195 196 197 198 199 202 203 205 207
 208 211 213 216 218 219 220 221 223 224 226 227 228 231 232 234 235 238
 239 241 242 243 245 246 247 248 250 251 253 254 256 259 262 264 266 268
 269 270 273 274 275 277 279 281 282 284 285 288 289 290 291 292 293 294
 296 298]
CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.29 µs




fold 3
[  3   6   7   8   9  10  12  13  17  18  20  21  22  23  25  28  30  31
  33  35  37  38  41  43  44  45  48  49  51  52  55  56  58  59  62  63
  64  65  66  68  69  70  71  72  73  74  75  76  77  78  80  81  82  83
  84  85  86  88  89  90  91  92  93  94  95  96 100 101 102 103 105 106
 107 108 111 113 115 117 118 121 122 123 126 128 130 131 132 133 134 135
 138 139 140 144 145 146 147 148 149 150 151 152 154 157 158 160 163 164
 165 167 168 169 170 171 173 174 175 177 179 180 181 182 184 185 186 187
 188 189 191 193 195 199 200 201 202 204 205 206 209 210 211 212 214 215
 217 219 220 221 222 223 225 226 228 229 230 232 233 235 236 237 238 240
 241 243 244 247 249 251 252 253 254 255 257 258 260 261 262 263 265 267
 270 271 272 276 278 280 283 284 286 287 288 289 290 291 293 295 296 297
 298 299]
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [12]:
def create_custome_sentencebert(model):
    """
    adds a pooler at the end of the fine-tuned model
    """
    bert_model_s = model
    pooling_model = models. Pooling(
        768,
        pooling_mode_mean_tokens = True,
        pooling_mode_cls_token = False,
        pooling_mode_max_tokens = False,
    )
    model = SentenceTransformer(modules= [bert_model_s, pooling_model])
    return model

sentence_bert_embedder = create_custome_sentencebert(model = model)

# Creating Embeddings

In [13]:
%%time
# creating corpus embeddings

corpus_embeddings = sentence_bert_embedder.encode(corpora,
                                                  convert_to_tensor=True,
                                                  show_progress_bar=True,
                                                  batch_size = 256)


CPU times: user 12.3 s, sys: 9.54 s, total: 21.8 s
Wall time: 20.6 s


# Sample Qustions

In [ ]:
# creating sample question embeddings

question = str(questions[:1])
question_emb = sentence_bert_embedder.encode(question,
                               convert_to_tensor=False,
                               show_progress_bar = False)

Searching with FAISS

In [ ]:
## Similarity Scores with FAISS

index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
# corpus
emb_cor = np.array(corpus_embeddings)
faiss.normalize_L2(emb_cor)

# query
index.add(emb_cor)
emb_que = np.array([question_emb])
faiss.normalize_L2(emb_que)

top_k = index.search(emb_que, 100)

In [ ]:
sentence_bert_WikiNli_indices = []
sentence_bert_WikiNli_scores = []

print(question)
print('Top results!')
for score, idx in zip(top_k[0][0], top_k[1][0]):
    print(colored(f'{idx}th', 'blue'),' corpus with score', colored( f'{score:.2f}:\n', 'blue'),  corpora[idx])
    
    sentence_bert_WikiNli_indices.append(idx)
    sentence_bert_WikiNli_scores.append(score)

In [ ]:
question

# Sampling in Batches

In [14]:
questions = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Sample Queries/Titles_with_Corona.pkl')

Don't put faiss itself in the function it'll give wrong results and indices

In [15]:
index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
emb_cor = np.array([i.numpy() for i in corpus_embeddings])
faiss.normalize_L2(emb_cor)
index.add(emb_cor)



In [16]:
results = get_resutls(questions, top_n = 50)

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
آیا زنان باردار باید برای کروناآزمایش شوند
آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
آیا در صورت مبتلا شدن به کووید۱۹ می‌توانم کودک خود را در آغوش بگیرم
من به کرونامبتلا هستم حال عمومی مساعدی ندارم که بتوانم به کودک خود مستقیما شیر بدهم چه باید بکنم
آیا خشک­ کن­های دست در نابودی ویروس جدید کرونا موثر هستند
آیا گندزداییضدعفونی با لامپ ماورای بنفش ویروس جدید کرونا را از بین می­برد
آیا سنجش حرارت بدن اندازه گیری تب در شنسایی افراد مبتلا به عفونت ویروس جدید کرونا موثر است
آیا اسپری کردن الکل یا محلول‌های حاوی کلر روی تمام بدن می­تواند ویروس جدید کرونا را نابود کند
آیا واکسنهای ذات­الریه در برابر ویروس جدید کرونا موثر هستند
آیا ش

# Saving all in a pickle

In [18]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/sbert-WikiNli-FineTuned_300_2.pkl', 'wb') as f:
    pickle.dump(results, f)